In [ ]:
import librosa
import numpy as np
import soundfile as sf
import torch

from IPython.display import Audio, display
from os import path
from torch.autograd import Variable

from NeuralStyleTransfer import CNNModel, GramMatrix, StyleLoss
from utils import plot_spectrum, read_audio_spectum, run_style_transfer

DURATION = 30
N_FFT = 2048
SAMPLE_RATE = 22050

In [ ]:
content_fname = "johntejada-1"
style_fname = "beat-box-2"

content_path = path.join("./wavs/corpus", content_fname + ".wav")
style_path = path.join("./wavs/target", style_fname + ".wav")

In [ ]:
style_audio, style_sr = read_audio_spectum(style_path, duration=DURATION, n_fft=N_FFT, sr=SAMPLE_RATE)
content_audio, content_sr = read_audio_spectum(content_path, duration=DURATION, n_fft=N_FFT, sr=SAMPLE_RATE)

plot_spectrum(style_audio)
display(Audio(style_path))

plot_spectrum(content_audio)
display(Audio(content_path))

In [ ]:
num_samples = min(style_audio.shape[1], content_audio.shape[1])
num_channels = min(style_audio.shape[0], content_audio.shape[0])

style_audio = style_audio[:num_channels, :num_samples].reshape([1, num_channels, num_samples])
content_audio = content_audio[:num_channels, :num_samples].reshape([1, num_channels, num_samples])

print(style_audio.shape, content_audio.shape)

In [ ]:
cnn = CNNModel()

if torch.cuda.is_available():
    cnn = cnn.cuda()
    style_float = Variable((torch.from_numpy(style_audio)).cuda())
    content_float = Variable((torch.from_numpy(content_audio)).cuda())	
else:
    style_float = Variable(torch.from_numpy(style_audio))
    content_float = Variable(torch.from_numpy(content_audio))

In [ ]:
input_float = content_float.clone()

num_steps = 2500
style_weight = 2500

output = run_style_transfer(
    cnn,
    style_float,
    input_float,
    num_steps=num_steps,
    style_weight=style_weight,
    style_layers=['conv_1'],
    lr=0.03)

if torch.cuda.is_available():
    output = output.cpu()

In [ ]:
output_np = output.squeeze(0).numpy()

a = np.zeros_like(output_np)
a = np.exp(output_np) - 1

# This code is supposed to do phase reconstruction
p = 2 * np.pi * np.random.random_sample(a.shape) - np.pi
for i in range(500):
    S = a * np.exp(1j * p)
    x = librosa.istft(S)
    p = np.angle(librosa.stft(x, n_fft=N_FFT))

output_fname = f'output1D-4096_c-{content_fname}_s-{style_fname}_iter-{num_steps}_sw-{style_weight}_k3s1p1.wav'
print(output_fname)
sf.write(path.join("./out", output_fname), x, style_sr)